In [1]:
import datetime as dt
import json
import re
import time

import datazimmer as dz
import pandas as pd

from aswan import get_soup, get_json, add_url_params, ConnectionError
from tqdm import tqdm

In [2]:
fifa_rank_base = dz.SourceUrl("https://www.fifa.com/fifa-world-ranking/men")

2023-06-22 16:45.06 [warning  ] can't get module name, likely due to notebook call


In [3]:
soup = get_soup(fifa_rank_base)

In [4]:
dr = '"dates"\:\[.*?\]'

In [5]:
date_ids = json.loads("{" + re.findall(dr, str(soup))[0] + "}")

In [6]:
ranks_by_date = {}
for date_dic in tqdm(date_ids["dates"]):
    did = date_dic["id"]
    ranks_by_date[did] = get_json(
        "https://www.fifa.com/api/ranking-overview",
        params={"locale": "en", "dateId": date_dic["id"]},
    )
    time.sleep(0.1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 324/324 [01:31<00:00,  3.53it/s]


In [7]:
uefa_seasons = list(range(2003, dt.date.today().year + 1))

In [8]:
outer_keys = ["countryName", "displayName"]

all_ranks = []
for season_year in tqdm(uefa_seasons):
    for pref in [
        "coefficientType=MEN_CLUB&coefficientRange=OVERALL",
        "coefficientType=MEN_ASSOCIATION&coefficientRange=SEASONAL",
        "coefficientType=MEN_ASSOCIATION&coefficientRange=OVERALL",
    ]:
        url = (
            f"https://comp.uefa.com/v2/coefficients?{pref}&"
            f"seasonYear={season_year}&page=1&pagesize=500&language=en"
        )

        try:
            club_json = get_json(url)
        except ConnectionError:
            continue

        for member in club_json["data"]["members"]:
            outer_d = {k: member["member"][k] for k in outer_keys}
            for ranks in member["seasonRankings"]:
                all_ranks.append(outer_d | ranks)
    time.sleep(1)

  0%|                                                                                                                                                                                                               | 0/21 [00:00<?, ?it/s]

2023-06-22 16:46.40 [warning  ] Failed Try                     attempt=0 e_msg=404 e_type=int handler=RequestJsonHandler proxy=None status=RETRY url=https://comp.uefa.com/v2/coefficients?coefficientType=MEN_CLUB&coefficientRange=OVERALL&seasonYear=2003&page=1&pagesize=500&language=en
2023-06-22 16:46.40 [warning  ] Failed Try                     attempt=1 e_msg=404 e_type=int handler=RequestJsonHandler proxy=None status=RETRY url=https://comp.uefa.com/v2/coefficients?coefficientType=MEN_CLUB&coefficientRange=OVERALL&seasonYear=2003&page=1&pagesize=500&language=en


  5%|█████████▍                                                                                                                                                                                             | 1/21 [00:02<00:49,  2.48s/it]

2023-06-22 16:46.42 [warning  ] Failed Try                     attempt=0 e_msg=404 e_type=int handler=RequestJsonHandler proxy=None status=RETRY url=https://comp.uefa.com/v2/coefficients?coefficientType=MEN_CLUB&coefficientRange=OVERALL&seasonYear=2004&page=1&pagesize=500&language=en
2023-06-22 16:46.43 [warning  ] Failed Try                     attempt=1 e_msg=404 e_type=int handler=RequestJsonHandler proxy=None status=RETRY url=https://comp.uefa.com/v2/coefficients?coefficientType=MEN_CLUB&coefficientRange=OVERALL&seasonYear=2004&page=1&pagesize=500&language=en


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:31<00:00,  1.52s/it]


In [11]:
pd.DataFrame(all_ranks).drop_duplicates().to_csv(dz.get_raw_data_path("uefa-ranks.csv"), index=None)

In [12]:
date_dic = {d["id"]: d["text"] for d in date_ids["dates"]}

In [13]:
todrop = ["flag", "tag"]

In [14]:
def flat(d0):
    d = d0.copy()
    ri = d.pop("rankingItem")
    return {k: v for k, v in (ri | d).items() if k not in todrop}

In [19]:
fifa_dfs = []
for k, v in ranks_by_date.items():
    fifa_dfs.append(pd.DataFrame(map(flat, v["rankings"] or [])).assign(date=date_dic[k]))

In [23]:
pd.concat(fifa_dfs, ignore_index=True).drop_duplicates().to_csv(
    dz.get_raw_data_path("fifa-ranks.csv"), index=None
)